<a href="https://colab.research.google.com/github/PabloAndree/Python_PA/blob/main/UTEC_ML_Caso2_14112024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso 2: Mantenimiento predictivo

## Librerías

In [27]:
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [28]:
from sklearn import preprocessing # Librería para realizar pre-procesamiento

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

## Lectura de datos

In [30]:
ruta_in = "/gdrive/My Drive/"
ruta_out = "/gdrive/My Drive/"

In [31]:
data = pd.read_csv(ruta_in + "ai4i2020 (1).csv")

In [32]:
data.head(3)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0


## Preparación de los datos

In [33]:
print(f"La cantidad de filas y columnas son {data.shape[0]}  y {data.shape[1]} respectivamente")

La cantidad de filas y columnas son 10000  y 14 respectivamente


In [34]:
res_var_type = data.groupby(["Type"]).agg({"UDI":"count"}).reset_index().rename(columns={"Type":"type",
                                                                          "UDI":"nro_reg"})

In [35]:
res_var_type

,type,nro_reg
0,H,1003
1,L,6000
2,M,2997


## Label Encoding

In [36]:
l_encoder = preprocessing.LabelEncoder()

data["Type_le"] = l_encoder.fit_transform(data["Type"])

In [37]:
data.head(3)

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_le
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,2
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,1
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,1


## One Hot Encoding

In [38]:
data_ = pd.get_dummies(data,columns=["Type"])
data_["Type_H"] = data_["Type_H"]*1
data_["Type_L"] = data_["Type_L"]*1
data_["Type_M"] = data_["Type_M"]*1
data_.head(3)

,UDI,Product ID,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_le,Type_H,Type_L,Type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,2,0,0,1
1,2,L47181,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,1,0,1,0
2,3,L47182,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,1,0,1,0


In [39]:
data_.columns.values

array(['UDI', 'Product ID', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF', 'Type_le', 'Type_H', 'Type_L', 'Type_M'], dtype=object)

In [40]:
# Crear listas de variables
var_id = ['UDI', 'Product ID']
var_feat_1 = ['Air temperature [K]','Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]','Type_le'] # Features usando mét Label Encoder
var_feat_2 = ['Air temperature [K]','Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]','Type_H', 'Type_L'] # Features usando mét One Hot Encoder
var_target = ['Machine failure']

In [41]:
data_[var_feat_1].head(3)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_le
0,298.1,308.6,1551,42.8,0,2
1,298.2,308.7,1408,46.3,3,1
2,298.1,308.5,1498,49.4,5,1


In [42]:
data_[var_target].head(3)

,Machine failure
0,0
1,0
2,0


## Escalamiento de variables

In [43]:
data_.head(2)

,UDI,Product ID,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_le,Type_H,Type_L,Type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,2,0,0,1
1,2,L47181,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,1,0,1,0


In [44]:
px.histogram(data_,x="Rotational speed [rpm]",title="Histograma - Rotational speed")

In [45]:
# Definir la forma de escalar las variables
sc_t = StandardScaler()

var_transf = ["Rotational speed [rpm]"]
ct = ColumnTransformer([('Standard_Scaler',sc_t,var_transf)],remainder='passthrough')

In [46]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('Standard_Scaler', StandardScaler(),
                                 ['Rotational speed [rpm]'])])

In [47]:
data_st1 = ct.fit_transform(data_[var_feat_1])
data_st1

array([[ 6.81851403e-02,  2.98100000e+02,  3.08600000e+02,
         4.28000000e+01,  0.00000000e+00,  2.00000000e+00],
       [-7.29471505e-01,  2.98200000e+02,  3.08700000e+02,
         4.63000000e+01,  3.00000000e+00,  1.00000000e+00],
       [-2.27449840e-01,  2.98100000e+02,  3.08500000e+02,
         4.94000000e+01,  5.00000000e+00,  1.00000000e+00],
       ...,
       [ 5.92518879e-01,  2.99000000e+02,  3.08600000e+02,
         3.34000000e+01,  2.20000000e+01,  2.00000000e+00],
       [-7.29471505e-01,  2.99000000e+02,  3.08700000e+02,
         4.85000000e+01,  2.50000000e+01,  0.00000000e+00],
       [-2.16293803e-01,  2.99000000e+02,  3.08700000e+02,
         4.02000000e+01,  3.00000000e+01,  2.00000000e+00]])

In [48]:
data_[var_feat_1].head(3)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_le
0,298.1,308.6,1551,42.8,0,2
1,298.2,308.7,1408,46.3,3,1
2,298.1,308.5,1498,49.4,5,1


In [49]:
feat = var_transf + [i for i in data_[var_feat_1].columns if i not in var_transf]
feat

['Rotational speed [rpm]',
 'Air temperature [K]',
 'Process temperature [K]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Type_le']

In [50]:
data_st1 = pd.DataFrame(data_st1,index=data_.index,columns = feat)

In [51]:
data_st1.head(2)

,Rotational speed [rpm],Air temperature [K],Process temperature [K],Torque [Nm],Tool wear [min],Type_le
0,0.068185,298.1,308.6,42.8,0.0,2.0
1,-0.729472,298.2,308.7,46.3,3.0,1.0


In [52]:
data_[var_feat_1].head(2)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_le
0,298.1,308.6,1551,42.8,0,2
1,298.2,308.7,1408,46.3,3,1


In [55]:
# Definir la forma de escalar las variables
sc_t2 = MinMaxScaler()

var_transf2 = ["Rotational speed [rpm]"]
ct2 = ColumnTransformer([('MinMax_Scaler',sc_t2,var_transf2)],remainder='passthrough')

# Transformamos el DataFrame original
data_st2 = ct2.fit_transform(data_[var_feat_1])
data_st2

# Identificar el orden de aparación de las variables después del escalamiento
feat2 = var_transf2 + [i for i in data_[var_feat_1].columns if i not in var_transf2]
feat2

# Convertir el arreglo de arreglos resultante del escalamiento en un DataFrame
data_st2 = pd.DataFrame(data_st2,index=data_.index,columns = feat2)

# Mostrar los primeros registros del DataFrame original y el escalado

print(f"Los primeros registros del DataFrame original \n{data_[var_feat_1].head(3)}\n")
print(f"Los primeros registros del DataFrame escalado \n{data_st2.head(3)}\n")

# Mostrar los histogramas de la variable sin transformar y de la variable escalada
px.histogram(data_st2,x="Rotational speed [rpm]",title="Histograma - Rotational speed (escalada)")

Los primeros registros del DataFrame original 
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0                298.1                    308.6                    1551   
1                298.2                    308.7                    1408   
2                298.1                    308.5                    1498   

   Torque [Nm]  Tool wear [min]  Type_le  
0         42.8                0        2  
1         46.3                3        1  
2         49.4                5        1  

Los primeros registros del DataFrame escalado 
   Rotational speed [rpm]  Air temperature [K]  Process temperature [K]  \
0                0.222934                298.1                    308.6   
1                0.139697                298.2                    308.7   
2                0.192084                298.1                    308.5   

   Torque [Nm]  Tool wear [min]  Type_le  
0         42.8              0.0      2.0  
1         46.3              3.0      1.0  
2 

In [56]:
# Definir la forma de escalar las variables
sc_t3 = StandardScaler()

var_transf3 = ['Air temperature [K]', 'Process temperature [K]','Rotational speed [rpm]']
ct3 = ColumnTransformer([('Standard_Scaler',sc_t3,var_transf3)],remainder='passthrough')

# Transformamos el DataFrame original
data_st3 = ct3.fit_transform(data_[var_feat_1])

# Identificar el orden de aparación de las variables después del escalamiento
feat3 = var_transf3 + [i for i in data_[var_feat_1].columns if i not in var_transf3]
feat3

# Convertir el arreglo de arreglos resultante del escalamiento en un DataFrame
data_st3 = pd.DataFrame(data_st3,index=data_.index,columns = feat3)

# Mostrar los primeros registros del DataFrame original y el escalado

print(f"Los primeros registros del DataFrame original \n{data_[var_feat_1].head(3)}\n")
print(f"Los primeros registros del DataFrame escalado \n{data_st3.head(3)}\n")

#################################################################################################




Los primeros registros del DataFrame original 
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0                298.1                    308.6                    1551   
1                298.2                    308.7                    1408   
2                298.1                    308.5                    1498   

   Torque [Nm]  Tool wear [min]  Type_le  
0         42.8                0        2  
1         46.3                3        1  
2         49.4                5        1  

Los primeros registros del DataFrame escalado 
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0            -0.952389                -0.947360                0.068185   
1            -0.902393                -0.879959               -0.729472   
2            -0.952389                -1.014761               -0.227450   

   Torque [Nm]  Tool wear [min]  Type_le  
0         42.8              0.0      2.0  
1         46.3              3.0      1.0  
2 

In [54]:
# Definir la forma de escalar las variables
sc_t4 = MinMaxScaler()

var_transf4 = ['Air temperature [K]', 'Process temperature [K]','Rotational speed [rpm]']
ct4 = ColumnTransformer([('MinMax_Scaler',sc_t4,var_transf4)],remainder='passthrough')

# Transformamos el DataFrame original
data_st4= ct4.fit_transform(data_[var_feat_1])

# Identificar el orden de aparación de las variables después del escalamiento
feat4 = var_transf4 + [i for i in data_[var_feat_1].columns if i not in var_transf4]
feat4

# Convertir el arreglo de arreglos resultante del escalamiento en un DataFrame
data_st4 = pd.DataFrame(data_st4,index=data_.index,columns = feat4)

# Mostrar los primeros registros del DataFrame original y el escalado

print(f"Los primeros registros del DataFrame original \n{data_[var_feat_1].head(3)}\n")
print(f"Los primeros registros del DataFrame escalado \n{data_st4.head(3)}\n")

Los primeros registros del DataFrame original 
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0                298.1                    308.6                    1551   
1                298.2                    308.7                    1408   
2                298.1                    308.5                    1498   

   Torque [Nm]  Tool wear [min]  Type_le  
0         42.8                0        2  
1         46.3                3        1  
2         49.4                5        1  

Los primeros registros del DataFrame escalado 
   Air temperature [K]  Process temperature [K]  Rotational speed [rpm]  \
0             0.304348                 0.358025                0.222934   
1             0.315217                 0.370370                0.139697   
2             0.304348                 0.345679                0.192084   

   Torque [Nm]  Tool wear [min]  Type_le  
0         42.8              0.0      2.0  
1         46.3              3.0      1.0  
2 

In [57]:
data_5 = data_[var_feat_1]
data_5.head(3)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_le
0,298.1,308.6,1551,42.8,0,2
1,298.2,308.7,1408,46.3,3,1
2,298.1,308.5,1498,49.4,5,1


In [58]:
data_5["B_Rotational_speed"] =pd.qcut(data_5["Rotational speed [rpm]"],q=10)

data_5.head(3)

<ipython-input-58-3b73913a2a6c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type_le,B_Rotational_speed
0,298.1,308.6,1551,42.8,0,2,"(1541.0, 1584.0]"
1,298.2,308.7,1408,46.3,3,1,"(1405.0, 1439.0]"
2,298.1,308.5,1498,49.4,5,1,"(1470.0, 1503.0]"


In [59]:
data_5.groupby(["B_Rotational_speed"]).agg({"Type_le":"count"}).reset_index().rename(columns={"Type_le":"nro"})

<ipython-input-59-5c07d15b1935>:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,B_Rotational_speed,nro
0,"(1167.999, 1364.0]",1016
1,"(1364.0, 1405.0]",991
2,"(1405.0, 1439.0]",1015
3,"(1439.0, 1470.0]",996
4,"(1470.0, 1503.0]",990
5,"(1503.0, 1541.0]",1009
6,"(1541.0, 1584.0]",986
7,"(1584.0, 1644.0]",1006
8,"(1644.0, 1746.0]",994
9,"(1746.0, 2886.0]",997


Transformaciones matemáticas (Logarítmicas)

In [60]:
data_5["L_Rotational_speed"]= np.log(data_5["Rotational speed [rpm]"]+0.001)

<ipython-input-60-9c0984d84844>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

